In [2]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [3]:
joined_df = pd.read_csv('data/joined_data.csv')

industry_list = list(joined_df['Industry'].unique())
county_list = list(joined_df['County'].unique())
joined_df

,Year,Industry,County,WorkingPopulation,Population,EarningsEuro,pctIndustryEmployement
0,2002,"Agriculture, forestry and fishing",State,97281.0,3917203.0,80847.45,2.483430
1,2006,"Agriculture, forestry and fishing",State,89277.0,4239848.0,112227.39,2.105665
2,2011,"Agriculture, forestry and fishing",State,94247.0,4588252.0,114239.67,2.054094
3,2016,"Agriculture, forestry and fishing",State,89116.0,4761865.0,132583.20,1.871452
4,2002,"Agriculture, forestry and fishing",Carlow,1581.0,46014.0,809.06,3.435911
...,...,...,...,...,...,...,...
1723,2016,All industries,Donegal,58353.0,159192.0,3230.88,36.655736
1724,2002,All industries,Monaghan,21101.0,52593.0,914.77,40.121309
1725,2006,All industries,Monaghan,25378.0,55997.0,1296.83,45.320285
1726,2011,All industries,Monaghan,23005.0,60483.0,1274.55,38.035481


#Avail of a StandardScaler to get the numeric variables into the correct form for regressions analysis

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from numpy import mean, absolute, sqrt
from sklearn.model_selection import KFold, cross_val_score


def run_regression(target_variable, dataframe):
       
    catagorical_columns = ['Year', 'Industry', 'County']
    numeric_columns = ['WorkingPopulation', 'Population', 'EarningsEuro', 'pctIndustryEmployement']
    
    #We remove the target variable so that we can just give that list to the Pipeline
    numeric_columns.remove(target_variable)
    
    #Drop target value from the x dataframe
    X = dataframe.drop(target_variable, axis=1)
    X = dataframe.drop(catagorical_columns, axis=1)
    
    #Have the y dataframe only consist of the target value
    y = dataframe[[target_variable]]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.4, random_state=13, shuffle=True)

    """ct = ColumnTransformer([('Standard', StandardScaler(), numeric_columns),
                       ('OneHotEncoder', OneHotEncoder(), list(catagorical_columns))])"""
    
    ct = ColumnTransformer([('Standard', StandardScaler(), numeric_columns)])
    
    
    pipe= Pipeline([('transformer', ct), ('model', LinearRegression())])
    pipe.fit(X_train, y_train)
    score = pipe.score(X_test, y_test)
    print(f"The score for the model targeting {target_variable} is {score}")
    

regressions_df = joined_df


"""for outer_index, industry in enumerate(industry_list):
    print(f'Filtering by {industry}')
    regressions_df = joined_df[joined_df['Industry']==industry].reset_index(drop=True)"""
for inner_index, target_variable in enumerate(['WorkingPopulation', 'Population', 'EarningsEuro', 'pctIndustryEmployement']):
    run_regression(target_variable, joined_df)

The score for the model targeting WorkingPopulation is 0.22660135852560648
The score for the model targeting Population is 0.9829131045444601
The score for the model targeting EarningsEuro is 0.9827333725211834
The score for the model targeting pctIndustryEmployement is 0.12345981743521206
